In [1]:
## Imports
from __future__ import print_function

import torch
from torch import optim
from torch.utils.data import DataLoader
import torch.nn as nn

from torchvision import transforms, datasets, models

from unet.unet_model import RegressorNet
# from unet.unet_model import RegressorUNet
from unet.unet_parts import SegmentationRegressionLoss

from utils.dataset import CrossValDataset
from utils.model_eval import dice_coefficient
from utils.data import RegressorDataset

from torch.utils.tensorboard import SummaryWriter

import numpy as np

In [2]:
## Device
if torch.cuda.is_available():
    device = 'cuda'
    print('Using GPU')
else:
    device = 'cpu'
    print('Using CPU')

Using GPU


In [5]:
## Dataset
groupPath  = "D:/autopos/train"
originPath = "D:/autopos/train/origin.csv"
batch_size = 16

## Training
n_channels = 1
n_classes = 13
n_regress = 2
# n_bottle = 256
n_hidden = 256
# n_hidden = [512,256,128,64,32,16]
# n_hidden = [512,256]
# n_hidden = [1024,1024,512,512,256,256]
# n_hidden = [256,256,512,512,1024,1024]
img_size = 512

learning_rate = 1e-3
weight_decay  = 1e-10
momentum      = 0.9

num_epochs    = 41

##Cropping
D = 350
N = 512
xc=0
yc=0
centermode = 'rand'
mu = np.array([0,0])
sigma= 50

In [8]:
## Train
fullDirList = np.array([groupPath+"/group_00",groupPath+"/group_01",groupPath+"/group_02",groupPath+"/group_03",groupPath+"/group_04"])
inds = np.arange(len(fullDirList))
# for partition in range(len(fullDirList)):
for partition in range(1):
    trainInd = np.delete(inds,[partition])
    testInd  = inds[partition]
    trainDirs = fullDirList[trainInd]
    testDir   = [fullDirList[testInd]]
    
    trainDataset = RegressorDataset([x + "/img" for x in trainDirs], [x + "/seg" for x in trainDirs], originPath,D=D,N=N,xc=xc,yc=yc,centermode=centermode,mu=mu,sigma=sigma)
    testDataset  = RegressorDataset([x + "/img" for x in testDir],   [x + "/seg" for x in testDir],   originPath,D=D,N=N,xc=xc,yc=yc,centermode=centermode,mu=mu,sigma=sigma)
    
    trainLoader  = DataLoader(trainDataset,batch_size=batch_size)
    testLoader   = DataLoader(testDataset,batch_size=batch_size)
    
    # Define Net
    net = models.vgg16(pretrained=True)
    
    for param in net.parameters():
        param.requires_grad = False
    
    n_inputs = net.classifier[6].in_features
    net.classifier[6] = nn.Sequential(
        nn.Linear(n_inputs, n_hidden), nn.ReLU(), nn.Dropout(0.4),
        nn.Linear(n_hidden, n_regress))
    
    print(net)
    total_params = sum(p.numel() for p in net.parameters())
    print(f'{total_params:,} total parameters.')
    total_trainable_params = sum(
        p.numel() for p in net.parameters() if p.requires_grad)
    print(f'{total_trainable_params:,} training parameters.')
    
#     net = RegressorUNet(n_channels=n_channels, n_classes=n_classes,n_regress=n_regress,n_hidden=n_hidden,n_bottle=n_bottle,img_size=img_size)
#     net.to(device=device)
#     net.apply_state_dict(pretrained_model)
    
#     net = RegressorNet(n_channels=n_channels, n_regress=n_regress,n_hidden=n_hidden,n_bottle=n_bottle,img_size=img_size)
#     net = RegressorNet(n_channels=n_channels, n_regress=n_regress,n_hidden=n_hidden,img_size=img_size)
#     print(net)
#     net.apply_state_dict(pretrained_model)
#     net.to(device=device)
    
    
#     print("Successfully loaded pretrained model")
    
    # Optimizer
    optimizer  = optim.RMSprop(net.parameters(),lr = learning_rate, weight_decay = weight_decay, momentum = momentum)
#     critereon  = SegmentationRegressionLoss()  
    critereon = nn.MSELoss()
#     lossweight = 100 
    
    #Tensorboard
    writer = SummaryWriter('runs/regress_modeldeep2_partition_'+str(partition))
    
    for epoch in range(num_epochs):
        net.train()
        epoch_train_loss = 0
        for trainIndex,batch in enumerate(trainLoader,0):
            imgs = batch['img']
#             segs_gt = batch['seg']
            regs_gt = batch['org']
            
            imgs = imgs.to(device=device, dtype=torch.float32)
#             segs_gt = segs_gt.to(device=device, dtype = torch.long)
            regs_gt = regs_gt.to(device=device, dtype = torch.float32)
            
#             segs_pr,regs_pr = net(imgs)
            regs_pr = net(imgs)
            
#             loss = critereon(segs_pr,regs_pr,segs_gt,regs_gt,lossweight)
            loss = critereon(regs_pr,regs_gt)
            
            epoch_train_loss += loss.item()/len(trainLoader)
            
            regs_err = regs_pr-regs_gt
            print(regs_err)
            
            optimizer.zero_grad()
            loss.backward()
#             nn.utils.clip_grad_value_(net.parameters(),0.1)
            optimizer.step()
            
            print("Train: " + str(loss.item()))
            
        writer.add_scalar('Training Loss',
                        epoch_train_loss,
                        epoch)
        
        net.eval()
        epoch_test_loss = 0
        for testIndex,batch in enumerate(testLoader,0):
            imgs = batch['img']
#             segs_gt = batch['seg']
            regs_gt = batch['org']
            
            imgs = imgs.to(device=device, dtype=torch.float32)
#             segs_gt = segs_gt.to(device=device, dtype = torch.long)
            regs_gt = regs_gt.to(device=device, dtype = torch.float32)
            
#             segs_pr,regs_pr = net(imgs)
            regs_pr = net(imgs)
            
#             loss = critereon(segs_pr,regs_pr,segs_gt,regs_gt,lossweight)
            loss = critereon(regs_pr,regs_gt)
    
            epoch_test_loss += loss.item()/len(testLoader)
            
            regs_err = regs_pr-regs_gt
            print(regs_err)
            
            print("Test: " + str(loss.item()))
            
        writer.add_scalar('Validation Loss',
                        epoch_test_loss,
                        epoch)
        
        savefile = "D:\\autoseg\\nets\\unet-save" + str(partition) + "_" + str(epoch) + ".pth"
        torch.save(net.state_dict(),savefile)


VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

RuntimeError: Given groups=1, weight of size [64, 3, 3, 3], expected input[16, 1, 512, 512] to have 3 channels, but got 1 channels instead

In [ ]:
for param in net.parameters():
    print(param)